### Imports 

In [1]:
from azure.communication.chat import ChatClient, CommunicationTokenCredential, ChatMessageType,ChatParticipant
from azure.communication.identity import CommunicationIdentityClient, CommunicationUserIdentifier
from datetime import datetime
from datetime import datetime, timedelta
import os
import requests
import json
import openai
from dotenv import load_dotenv

## Connection Strings

In [2]:
# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")
## Azure Communication Service
COMMUNICATION_CONNECTION_STRING = os.getenv("COMMUNICATION_CONNECTION_STRING")
COMMUNICATION_ENDPOINT = os.getenv("COMMUNICATION_ENDPOINT")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

load_dotenv()


True

In [3]:
client = CommunicationIdentityClient.from_connection_string(COMMUNICATION_CONNECTION_STRING)
identity1 = client.create_user()
token_result1 = client.get_token(identity1, ["chat"])
identity2 = client.create_user()
token_result2 = client.get_token(identity2, ["chat"])


## Configuring the chat client

In [4]:
Agent  = ChatParticipant(identifier=identity1, display_name="Agent", share_history_time=datetime.utcnow()) 
Customer  = ChatParticipant(identifier=identity2, display_name="Customer", share_history_time=datetime.utcnow())
participants = [Agent, Customer ]

chat_client1 = ChatClient(COMMUNICATION_ENDPOINT, CommunicationTokenCredential(token_result1.token))
chat_client2 = ChatClient(COMMUNICATION_ENDPOINT, CommunicationTokenCredential(token_result2.token))
topic="Support conversation"
create_chat_thread_result = chat_client1.create_chat_thread(topic, thread_participants=participants)
chat_thread_client1 = chat_client1.get_chat_thread_client(create_chat_thread_result.chat_thread.id)
chat_thread_client2 = chat_client2.get_chat_thread_client(create_chat_thread_result.chat_thread.id)


C:\Users\VBALANTR\AppData\Local\Temp\ipykernel_20468\2455567013.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  Agent  = ChatParticipant(identifier=identity1, display_name="Agent", share_history_time=datetime.utcnow())
C:\Users\VBALANTR\AppData\Local\Temp\ipykernel_20468\2455567013.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  Customer  = ChatParticipant(identifier=identity2, display_name="Customer", share_history_time=datetime.utcnow())
Datetime with no tzinfo will be considered UTC.
Datetime with no tzinfo will be considered UTC.


## sample conversation between a customer support agent and a customer

In [5]:
agentText = [    
    "Thank you for reaching out to our customer support! How can I assist you with your food delivery today?",
    "I'm sorry to hear that. Could you provide me with your name and order number so I can check your details and assist you better?",
    "Thanks. Can you please share more details about the issue you're facing with your food delivery?",
    "I understand. It can be frustrating. Can you check if the delivery address is correct and accessible? Also, ensure that your contact details are up to date.",
    "Alright. Please verify the delivery address and ensure there are no obstacles for our delivery team. Let me know if the issue persists.",
    "That might be causing the delay. Can you confirm the delivery address and try placing the order again?",
    "Great to hear that! Is there anything else I can help you with regarding your food delivery?",
    "You're welcome. Feel free to reach out if you have any more questions. Enjoy your meal!"
]


In [6]:
customerText = [    
    "Hi, I'm having an issue with my food delivery. It hasn't arrived yet.",
    "Yes, my name is John, and my order number is.",
    "Well, it seems like my food hasn't been delivered on time. What could be the issue?",
    "I've checked the delivery address, and it's correct. Also, my phone is working fine.",
    "Okay, let me check. Yes, it seems like there's a small issue with the delivery address. Let me correct that.",
    "Sure, let me try that. Alright, I've updated the delivery address. Can you check the status now?",
    "No, that's all. Thanks for your help. Looking forward to my meal!",
    "Goodbye."
]

In [7]:
for x in range(len(agentText)):
    chat_thread_client1.send_message(content= agentText[x], sender_display_name="Agent", chat_message_type=ChatMessageType.TEXT)
    chat_thread_client2.send_message(content= customerText[x], sender_display_name="Customer", chat_message_type=ChatMessageType.TEXT)

## Generating a conversations transcript

In [8]:
start_time = datetime.utcnow() - timedelta(days=1)
messages = []

chat_messages = chat_thread_client1.list_messages(results_per_page=1, start_time=start_time)
for chat_message_page in chat_messages.by_page():
    for chat_message in chat_message_page:
        if(chat_message.type == ChatMessageType.TEXT):
            messages.append(chat_message)

# didn't know I had to filter out other messages

prompt = ""
for m in range(len(messages)-1, -1, -1):
    prompt = prompt + messages[m].sender_display_name + ": " + messages[m].content.message + "\n"
print(prompt)

C:\Users\VBALANTR\AppData\Local\Temp\ipykernel_20468\2785931093.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.utcnow() - timedelta(days=1)
Datetime with no tzinfo will be considered UTC.


Agent: Thank you for reaching out to our customer support! How can I assist you with your food delivery today?
Customer: Hi, I'm having an issue with my food delivery. It hasn't arrived yet.
Agent: I'm sorry to hear that. Could you provide me with your name and order number so I can check your details and assist you better?
Customer: Yes, my name is John, and my order number is.
Agent: Thanks. Can you please share more details about the issue you're facing with your food delivery?
Customer: Well, it seems like my food hasn't been delivered on time. What could be the issue?
Agent: I understand. It can be frustrating. Can you check if the delivery address is correct and accessible? Also, ensure that your contact details are up to date.
Customer: I've checked the delivery address, and it's correct. Also, my phone is working fine.
Agent: Alright. Please verify the delivery address and ensure there are no obstacles for our delivery team. Let me know if the issue persists.
Customer: Okay, le

## Summarize the chat thread

In [9]:

# Send a completion call to generate an answer
start_phrase = 'For the following conversation, extract a topic, summary, highlights (1-3 bullet points of key information) and the sentiment of both of the users.\n\n' + prompt
response = openai.Completion.create(engine=OPENAI_DEPLOYMENT_NAME, prompt=start_phrase, max_tokens=500)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase + '\n' + text)

For the following conversation, extract a topic, summary, highlights (1-3 bullet points of key information) and the sentiment of both of the users.

Agent: Thank you for reaching out to our customer support! How can I assist you with your food delivery today?
Customer: Hi, I'm having an issue with my food delivery. It hasn't arrived yet.
Agent: I'm sorry to hear that. Could you provide me with your name and order number so I can check your details and assist you better?
Customer: Yes, my name is John, and my order number is.
Agent: Thanks. Can you please share more details about the issue you're facing with your food delivery?
Customer: Well, it seems like my food hasn't been delivered on time. What could be the issue?
Agent: I understand. It can be frustrating. Can you check if the delivery address is correct and accessible? Also, ensure that your contact details are up to date.
Customer: I've checked the delivery address, and it's correct. Also, my phone is working fine.
Agent: Alrig